<a href="https://colab.research.google.com/github/ANEUP002/ANEUP002/blob/main/Medicare_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IOWA BUSINESS COMPETITION | WELLMARK BLUECROSS

## All important Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

##Monthly Enrollement Dataset January 2013 - December 2024

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_monthly = pd.read_csv('/content/drive/MyDrive/Wellmark Case/Medicare Cleaned Data/december_monthly_medicare_data (1).csv')

In [ ]:
df_bed_phys = pd.read_csv('/content/drive/MyDrive/Wellmark Case/Medicare Cleaned Data/yearly_bed_physician_data.csv')

In [ ]:
df_bed_phys.head(30)

,YEAR,STATE,BED_CNT,PHYSN_CNT
0,2022,IA,426944.0,7679.55
1,2022,SD,117600.0,4107.09
2,2024,IA,423499.0,7179.53
3,2024,SD,120879.0,4122.00
4,2013,IA,105768.0,1322.66
5,2013,SD,30263.0,646.38
6,2023,IA,427040.0,7344.87
7,2023,SD,120086.0,4124.63
8,2020,IA,423919.0,7519.30
9,2020,SD,120862.0,4001.89


In [ ]:
df_monthly.head()

,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,AGED_TOT_BENES,AGED_ESRD_BENES,AGED_NO_ESRD_BENES,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,MALE_TOT_BENES,FEMALE_TOT_BENES,WHITE_TOT_BENES,BLACK_TOT_BENES,API_TOT_BENES,HSPNC_TOT_BENES,NATIND_TOT_BENES,OTHR_TOT_BENES,AGE_LT_25_BENES,AGE_25_TO_44_BENES,AGE_45_TO_64_BENES,AGE_65_TO_69_BENES,AGE_70_TO_74_BENES,AGE_75_TO_79_BENES,AGE_80_TO_84_BENES,AGE_85_TO_89_BENES,AGE_90_TO_94_BENES,AGE_GT_94_BENES,DUAL_TOT_BENES,FULL_DUAL_TOT_BENES,PART_DUAL_TOT_BENES,NODUAL_TOT_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,A_TOT_BENES,A_ORGNL_MDCR_BENES,A_MA_AND_OTH_BENES,B_TOT_BENES,B_ORGNL_MDCR_BENES,B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES,PRSCRPTN_DRUG_DEEMED_ELIGIBLE_FULL_LIS_BENES,PRSCRPTN_DRUG_FULL_LIS_BENES,PRSCRPTN_DRUG_PARTIAL_LIS_BENES,PRSCRPTN_DRUG_NO_LIS_BENES
0,2013,January,State,IA,Iowa,Total,19,543983,466817,77166,471069,1942,469127,72914,1457,71457,243944,300039,520292,9040,3282,5586,702,5081,1171,16749,54994,118302,110319,87305,69056,50743,25841,9503,79994,63293,16701,463989,509882,432720,77162,543702,466540,77162,510163,432997,77166,398443,333050,65393,81084,6759,3648,306952
1,2013,January,State,SD,South Dakota,Total,46,145475,124873,20602,127444,516,126928,18031,466,17565,67633,77842,135609,576,482,829,6654,1325,255,4236,13540,33405,30826,23072,18187,12824,6606,2524,19584,12170,7414,125891,134779,114177,20602,145385,124783,20602,134869,114267,20602,96400,85363,11037,19905,2102,1037,73356
2,2013,February,State,IA,Iowa,Total,19,544301,467032,77269,470957,1895,469062,73344,1458,71886,244246,300055,520408,9083,3304,5625,703,5178,1210,16837,55297,120488,110079,86969,68624,50207,25330,9260,80066,63270,16796,464235,510036,432771,77265,544021,466756,77265,510316,433047,77269,398784,333264,65520,81283,6684,3641,307176
3,2013,February,State,SD,South Dakota,Total,46,145669,124872,20797,127543,504,127039,18126,467,17659,67748,77921,135789,577,484,836,6638,1345,256,4249,13621,34019,30768,23006,18101,12702,6484,2463,19585,12127,7458,126084,134912,114115,20797,145580,124783,20797,135001,114204,20797,96548,85478,11070,19981,2097,1039,73431
4,2013,March,State,IA,Iowa,Total,19,545485,468169,77316,471707,1862,469845,73778,1459,72319,244838,300647,521355,9124,3321,5671,705,5309,1244,16939,55595,122886,109899,86740,68299,49796,25013,9074,80264,63385,16879,465221,510875,433563,77312,545204,467892,77312,511156,433840,77316,399432,333866,65566,81730,6605,3615,307482


In [ ]:
df_monthly.columns

Index(['YEAR', 'MONTH', 'BENE_GEO_LVL', 'BENE_STATE_ABRVTN', 'BENE_STATE_DESC',
       'BENE_COUNTY_DESC', 'BENE_FIPS_CD', 'TOT_BENES', 'ORGNL_MDCR_BENES',
       'MA_AND_OTH_BENES', 'AGED_TOT_BENES', 'AGED_ESRD_BENES',
       'AGED_NO_ESRD_BENES', 'DSBLD_TOT_BENES',
       'DSBLD_ESRD_AND_ESRD_ONLY_BENES', 'DSBLD_NO_ESRD_BENES',
       'MALE_TOT_BENES', 'FEMALE_TOT_BENES', 'WHITE_TOT_BENES',
       'BLACK_TOT_BENES', 'API_TOT_BENES', 'HSPNC_TOT_BENES',
       'NATIND_TOT_BENES', 'OTHR_TOT_BENES', 'AGE_LT_25_BENES',
       'AGE_25_TO_44_BENES', 'AGE_45_TO_64_BENES', 'AGE_65_TO_69_BENES',
       'AGE_70_TO_74_BENES', 'AGE_75_TO_79_BENES', 'AGE_80_TO_84_BENES',
       'AGE_85_TO_89_BENES', 'AGE_90_TO_94_BENES', 'AGE_GT_94_BENES',
       'DUAL_TOT_BENES', 'FULL_DUAL_TOT_BENES', 'PART_DUAL_TOT_BENES',
       'NODUAL_TOT_BENES', 'A_B_TOT_BENES', 'A_B_ORGNL_MDCR_BENES',
       'A_B_MA_AND_OTH_BENES', 'A_TOT_BENES', 'A_ORGNL_MDCR_BENES',
       'A_MA_AND_OTH_BENES', 'B_TOT_BENES', 'B_ORGNL_MD

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Define a mapping for months to ensure correct ordering
month_order = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}

# Add a numerical month column for sorting
df_monthly['MONTH_NUM'] = df_monthly['MONTH'].map(month_order)

# Filter for IA and SD, and sort by YEAR and MONTH_NUM
df_filtered = df_monthly[df_monthly['BENE_STATE_ABRVTN'].isin(['IA', 'SD'])].sort_values(['YEAR', 'MONTH_NUM'])

# Get December data for each year (last month)
df_december = df_filtered[df_filtered['MONTH'] == 'December']

# Sum TOT_BENES and DUAL_TOT_BENES for IA and SD for each year
df_yearly = df_december.groupby('YEAR')[['TOT_BENES', 'DUAL_TOT_BENES']].sum().reset_index()

# Calculate YoY growth rates
df_yearly['TOT_BENES_GROWTH'] = df_yearly['TOT_BENES'].pct_change() * 100
df_yearly['DUAL_TOT_BENES_GROWTH'] = df_yearly['DUAL_TOT_BENES'].pct_change() * 100

# Fill NaN for first year with 0 (no previous year to compare)
df_yearly['TOT_BENES_GROWTH'] = df_yearly['TOT_BENES_GROWTH'].fillna(0)
df_yearly['DUAL_TOT_BENES_GROWTH'] = df_yearly['DUAL_TOT_BENES_GROWTH'].fillna(0)

# Create a figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add trace for total beneficiaries
fig.add_trace(
    go.Scatter(
        x=df_yearly['YEAR'],
        y=df_yearly['TOT_BENES'],
        mode='lines+markers+text',
        name='Total Beneficiaries (IA + SD)',
        marker=dict(color='blue', size=10),
        line=dict(color='blue', width=2),
        text=[f"{growth:.1f}%" for growth in df_yearly['TOT_BENES_GROWTH']],
        textposition='top center',
        textfont=dict(color='blue', size=10)
    ),
    secondary_y=False
)

# Add trace for dual total beneficiaries
fig.add_trace(
    go.Scatter(
        x=df_yearly['YEAR'],
        y=df_yearly['DUAL_TOT_BENES'],
        mode='lines+markers+text',
        name='Dual Total Beneficiaries (IA + SD)',
        marker=dict(color='green', size=10),
        line=dict(color='green', width=2),
        text=[f"{growth:.1f}%" for growth in df_yearly['DUAL_TOT_BENES_GROWTH']],
        textposition='top center',
        textfont=dict(color='green', size=10)
    ),
    secondary_y=True
)

# Update layout with titles and formatting
fig.update_layout(
    title='Total and Dual Total Beneficiaries with YoY Growth (IA + SD, 2013-2024)',
    title_font=dict(size=16),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5),
    hovermode="x unified",
    plot_bgcolor='white',
    xaxis=dict(
        title='Year',
        titlefont=dict(size=12),
        showgrid=True,
        gridcolor='lightgray',
        tickmode='array',
        tickvals=df_yearly['YEAR']
    ),
    yaxis=dict(
        title='Total Beneficiaries',
        titlefont=dict(color='blue', size=12),
        showgrid=True,
        gridcolor='lightgray',
        tickformat=',d'  # Format with commas
    ),
    yaxis2=dict(
        title='Dual Total Beneficiaries',
        titlefont=dict(color='green', size=12),
        showgrid=False,
        tickformat=',d'  # Format with commas
    )
)

# Add grid lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', secondary_y=False)

# Show the plot
fig.show()

# Print the yearly data for verification
print(df_yearly[['YEAR', 'TOT_BENES', 'DUAL_TOT_BENES', 'TOT_BENES_GROWTH', 'DUAL_TOT_BENES_GROWTH']])

    YEAR  TOT_BENES  DUAL_TOT_BENES  TOT_BENES_GROWTH  DUAL_TOT_BENES_GROWTH
0   2013     704641          101338          0.000000               0.000000
1   2014     721087          102474          2.333954               1.121001
2   2015     736229          102977          2.099885               0.490856
3   2016     754880          104376          2.533315               1.358556
4   2017     770963          104033          2.130537              -0.328620
5   2018     792368          105019          2.776398               0.947776
6   2019     809618          105522          2.177019               0.478961
7   2020     823277          109582          1.687092               3.847539
8   2021     838781          114510          1.883206               4.497089
9   2022     856135          119684          2.068955               4.518383
10  2023     873961          108781          2.082148              -9.109823
11  2024     889946          105523          1.829029              -2.995008

## Correlation Metrix

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Enhanced Correlation Analysis
def enhanced_correlation_analysis(df, target_columns):
    """
    Perform comprehensive correlation analysis with beautiful visualizations

    Parameters:
        df (pd.DataFrame): Input dataframe
        target_columns (list): List of target columns to analyze
    """

    # Step 1: Select numeric columns and handle missing values
    print("🔍 Preparing data for correlation analysis...")
    df_numeric = df.select_dtypes(include=[np.number])

    # Fill missing values with median (better than mean for skewed distributions)
    df_numeric = df_numeric.fillna(df_numeric.median())

    # Step 2: Calculate correlation matrix
    print("📊 Calculating correlation matrix...")
    correlation_matrix = df_numeric.corr()

    # Step 3: Create interactive heatmap
    print("🔥 Generating interactive heatmap...")
    fig_heatmap = px.imshow(
        correlation_matrix,
        color_continuous_scale='RdBu_r',
        zmin=-1,
        zmax=1,
        aspect="auto",
        title="Interactive Correlation Heatmap",
        labels=dict(color="Correlation"),
        height=800
    )

    # Add custom hover template
    fig_heatmap.update_traces(
        hovertemplate="<b>%{y}</b> vs <b>%{x}</b><br>Correlation: %{z:.2f}<extra></extra>"
    )

    # Rotate x-axis labels
    fig_heatmap.update_layout(
        xaxis=dict(tickangle=45),
        yaxis=dict(tickangle=0),
        margin=dict(l=100, r=100, b=150, t=100)
    )

    # Step 4: Create target-focused correlation plots
    print("🎯 Creating target-focused visualizations...")
    target_figs = []

    for target in target_columns:
        if target in correlation_matrix.columns:
            # Get top 10 positive and negative correlations
            corr_series = correlation_matrix[target].sort_values(ascending=False)
            top_pos = corr_series[1:11]  # Exclude self-correlation
            top_neg = corr_series[-10:]

            # Create figure
            fig = make_subplots(rows=1, cols=2,
                               subplot_titles=[
                                   f"Top 10 Positive Correlations with {target}",
                                   f"Top 10 Negative Correlations with {target}"
                               ])

            # Positive correlations
            fig.add_trace(
                go.Bar(
                    y=top_pos.index,
                    x=top_pos.values,
                    orientation='h',
                    marker=dict(color='#2ecc71'),
                    name='Positive',
                    hovertemplate="%{y}: %{x:.2f}<extra></extra>"
                ),
                row=1, col=1
            )

            # Negative correlations
            fig.add_trace(
                go.Bar(
                    y=top_neg.index,
                    x=top_neg.values,
                    orientation='h',
                    marker=dict(color='#e74c3c'),
                    name='Negative',
                    hovertemplate="%{y}: %{x:.2f}<extra></extra>"
                ),
                row=1, col=2
            )

            # Update layout
            fig.update_layout(
                title_text=f"Correlation Analysis for {target}",
                showlegend=False,
                height=600,
                margin=dict(l=150, r=150))

            fig.update_xaxes(range=[-1, 1], row=1, col=1)
            fig.update_xaxes(range=[-1, 1], row=1, col=2)

            target_figs.append(fig)

    # Step 5: Create correlation network for strongest relationships
    print("🕸️ Creating correlation network...")
    strong_correlations = correlation_matrix.stack().reset_index()
    strong_correlations.columns = ['Var1', 'Var2', 'Correlation']

    # Filter for strong correlations (absolute value > 0.7)
    strong_correlations = strong_correlations[
        (abs(strong_correlations['Correlation']) > 0.7) &
        (strong_correlations['Var1'] != strong_correlations['Var2'])
    ]

    if not strong_correlations.empty:
        fig_network = px.scatter(
            strong_correlations,
            x='Var1',
            y='Var2',
            size=abs(strong_correlations['Correlation']),
            color='Correlation',
            color_continuous_scale='RdBu_r',
            range_color=[-1, 1],
            hover_name='Var1',
            hover_data=['Var2', 'Correlation'],
            title="Strong Correlation Relationships (|r| > 0.7)"
        )

        fig_network.update_layout(
            xaxis_title="Variable 1",
            yaxis_title="Variable 2",
            height=700,
            margin=dict(l=100, r=100, b=150, t=100)
        )
    else:
        print("⚠️ No strong correlations (|r| > 0.7) found for network visualization")
        fig_network = None

    # Step 6: Create correlation distribution plot
    print("📈 Plotting correlation distribution...")
    corr_values = correlation_matrix.values.flatten()
    corr_values = corr_values[~np.isnan(corr_values)]  # Remove NaN values

    fig_dist = px.histogram(
        x=corr_values,
        nbins=50,
        title="Distribution of Correlation Coefficients",
        labels={'x': 'Correlation Coefficient'},
        color_discrete_sequence=['#3498db']
    )

    fig_dist.update_layout(
        bargap=0.1,
        xaxis=dict(range=[-1, 1]),
        yaxis_title="Count"
    )

    # Add mean line
    mean_corr = np.mean(corr_values)
    fig_dist.add_vline(
        x=mean_corr,
        line_dash="dash",
        line_color="red",
        annotation_text=f"Mean: {mean_corr:.2f}",
        annotation_position="top right"
    )

    # Step 7: Display all visualizations
    print("🎨 Rendering visualizations...")
    fig_heatmap.show()

    for fig in target_figs:
        fig.show()

    if fig_network:
        fig_network.show()

    fig_dist.show()

    # Return correlation matrix for further analysis
    return correlation_matrix

# Run the enhanced analysis
print("🚀 Starting enhanced correlation analysis")
target_columns = ["TOT_BENES", "DUAL_TOT_BENES"]
correlation_matrix = enhanced_correlation_analysis(df_monthly, target_columns)

🚀 Starting enhanced correlation analysis
🔍 Preparing data for correlation analysis...
📊 Calculating correlation matrix...
🔥 Generating interactive heatmap...
🎯 Creating target-focused visualizations...
🕸️ Creating correlation network...
📈 Plotting correlation distribution...
🎨 Rendering visualizations...


In [ ]:
# Create a new column combining YEAR and MONTH into a datetime object
df_monthly['Date'] = pd.to_datetime(df_monthly['YEAR'].astype(str) + '-' + df_monthly['MONTH'], format='%Y-%B')

# Check the result
print(df_monthly[['YEAR', 'MONTH', 'Date']].head())


   YEAR     MONTH       Date
0  2013   January 2013-01-01
1  2013   January 2013-01-01
2  2013  February 2013-02-01
3  2013  February 2013-02-01
4  2013     March 2013-03-01


In [ ]:
df_monthly.head()

,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,AGED_TOT_BENES,AGED_ESRD_BENES,AGED_NO_ESRD_BENES,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,MALE_TOT_BENES,FEMALE_TOT_BENES,WHITE_TOT_BENES,BLACK_TOT_BENES,API_TOT_BENES,HSPNC_TOT_BENES,NATIND_TOT_BENES,OTHR_TOT_BENES,AGE_LT_25_BENES,AGE_25_TO_44_BENES,AGE_45_TO_64_BENES,AGE_65_TO_69_BENES,AGE_70_TO_74_BENES,AGE_75_TO_79_BENES,AGE_80_TO_84_BENES,AGE_85_TO_89_BENES,AGE_90_TO_94_BENES,AGE_GT_94_BENES,DUAL_TOT_BENES,FULL_DUAL_TOT_BENES,PART_DUAL_TOT_BENES,NODUAL_TOT_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,A_TOT_BENES,A_ORGNL_MDCR_BENES,A_MA_AND_OTH_BENES,B_TOT_BENES,B_ORGNL_MDCR_BENES,B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES,PRSCRPTN_DRUG_DEEMED_ELIGIBLE_FULL_LIS_BENES,PRSCRPTN_DRUG_FULL_LIS_BENES,PRSCRPTN_DRUG_PARTIAL_LIS_BENES,PRSCRPTN_DRUG_NO_LIS_BENES,MONTH_NUM,Date
0,2013,January,State,IA,Iowa,Total,19,543983,466817,77166,471069,1942,469127,72914,1457,71457,243944,300039,520292,9040,3282,5586,702,5081,1171,16749,54994,118302,110319,87305,69056,50743,25841,9503,79994,63293,16701,463989,509882,432720,77162,543702,466540,77162,510163,432997,77166,398443,333050,65393,81084,6759,3648,306952,1,2013-01-01
1,2013,January,State,SD,South Dakota,Total,46,145475,124873,20602,127444,516,126928,18031,466,17565,67633,77842,135609,576,482,829,6654,1325,255,4236,13540,33405,30826,23072,18187,12824,6606,2524,19584,12170,7414,125891,134779,114177,20602,145385,124783,20602,134869,114267,20602,96400,85363,11037,19905,2102,1037,73356,1,2013-01-01
2,2013,February,State,IA,Iowa,Total,19,544301,467032,77269,470957,1895,469062,73344,1458,71886,244246,300055,520408,9083,3304,5625,703,5178,1210,16837,55297,120488,110079,86969,68624,50207,25330,9260,80066,63270,16796,464235,510036,432771,77265,544021,466756,77265,510316,433047,77269,398784,333264,65520,81283,6684,3641,307176,2,2013-02-01
3,2013,February,State,SD,South Dakota,Total,46,145669,124872,20797,127543,504,127039,18126,467,17659,67748,77921,135789,577,484,836,6638,1345,256,4249,13621,34019,30768,23006,18101,12702,6484,2463,19585,12127,7458,126084,134912,114115,20797,145580,124783,20797,135001,114204,20797,96548,85478,11070,19981,2097,1039,73431,2,2013-02-01
4,2013,March,State,IA,Iowa,Total,19,545485,468169,77316,471707,1862,469845,73778,1459,72319,244838,300647,521355,9124,3321,5671,705,5309,1244,16939,55595,122886,109899,86740,68299,49796,25013,9074,80264,63385,16879,465221,510875,433563,77312,545204,467892,77312,511156,433840,77316,399432,333866,65566,81730,6605,3615,307482,3,2013-03-01


In [ ]:
# Ensure that the 'YEAR' column is a unique index and sorted
df_monthly['Date'] = pd.to_datetime(df_monthly['Date'])
df_monthly.set_index('Date', inplace=True)

In [ ]:
df_monthly.head()

,YEAR,MONTH,BENE_GEO_LVL,BENE_STATE_ABRVTN,BENE_STATE_DESC,BENE_COUNTY_DESC,BENE_FIPS_CD,TOT_BENES,ORGNL_MDCR_BENES,MA_AND_OTH_BENES,AGED_TOT_BENES,AGED_ESRD_BENES,AGED_NO_ESRD_BENES,DSBLD_TOT_BENES,DSBLD_ESRD_AND_ESRD_ONLY_BENES,DSBLD_NO_ESRD_BENES,MALE_TOT_BENES,FEMALE_TOT_BENES,WHITE_TOT_BENES,BLACK_TOT_BENES,API_TOT_BENES,HSPNC_TOT_BENES,NATIND_TOT_BENES,OTHR_TOT_BENES,AGE_LT_25_BENES,AGE_25_TO_44_BENES,AGE_45_TO_64_BENES,AGE_65_TO_69_BENES,AGE_70_TO_74_BENES,AGE_75_TO_79_BENES,AGE_80_TO_84_BENES,AGE_85_TO_89_BENES,AGE_90_TO_94_BENES,AGE_GT_94_BENES,DUAL_TOT_BENES,FULL_DUAL_TOT_BENES,PART_DUAL_TOT_BENES,NODUAL_TOT_BENES,A_B_TOT_BENES,A_B_ORGNL_MDCR_BENES,A_B_MA_AND_OTH_BENES,A_TOT_BENES,A_ORGNL_MDCR_BENES,A_MA_AND_OTH_BENES,B_TOT_BENES,B_ORGNL_MDCR_BENES,B_MA_AND_OTH_BENES,PRSCRPTN_DRUG_TOT_BENES,PRSCRPTN_DRUG_PDP_BENES,PRSCRPTN_DRUG_MAPD_BENES,PRSCRPTN_DRUG_DEEMED_ELIGIBLE_FULL_LIS_BENES,PRSCRPTN_DRUG_FULL_LIS_BENES,PRSCRPTN_DRUG_PARTIAL_LIS_BENES,PRSCRPTN_DRUG_NO_LIS_BENES,MONTH_NUM
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-01-01,2013,January,State,IA,Iowa,Total,19,543983,466817,77166,471069,1942,469127,72914,1457,71457,243944,300039,520292,9040,3282,5586,702,5081,1171,16749,54994,118302,110319,87305,69056,50743,25841,9503,79994,63293,16701,463989,509882,432720,77162,543702,466540,77162,510163,432997,77166,398443,333050,65393,81084,6759,3648,306952,1
2013-01-01,2013,January,State,SD,South Dakota,Total,46,145475,124873,20602,127444,516,126928,18031,466,17565,67633,77842,135609,576,482,829,6654,1325,255,4236,13540,33405,30826,23072,18187,12824,6606,2524,19584,12170,7414,125891,134779,114177,20602,145385,124783,20602,134869,114267,20602,96400,85363,11037,19905,2102,1037,73356,1
2013-02-01,2013,February,State,IA,Iowa,Total,19,544301,467032,77269,470957,1895,469062,73344,1458,71886,244246,300055,520408,9083,3304,5625,703,5178,1210,16837,55297,120488,110079,86969,68624,50207,25330,9260,80066,63270,16796,464235,510036,432771,77265,544021,466756,77265,510316,433047,77269,398784,333264,65520,81283,6684,3641,307176,2
2013-02-01,2013,February,State,SD,South Dakota,Total,46,145669,124872,20797,127543,504,127039,18126,467,17659,67748,77921,135789,577,484,836,6638,1345,256,4249,13621,34019,30768,23006,18101,12702,6484,2463,19585,12127,7458,126084,134912,114115,20797,145580,124783,20797,135001,114204,20797,96548,85478,11070,19981,2097,1039,73431,2
2013-03-01,2013,March,State,IA,Iowa,Total,19,545485,468169,77316,471707,1862,469845,73778,1459,72319,244838,300647,521355,9124,3321,5671,705,5309,1244,16939,55595,122886,109899,86740,68299,49796,25013,9074,80264,63385,16879,465221,510875,433563,77312,545204,467892,77312,511156,433840,77316,399432,333866,65566,81730,6605,3615,307482,3


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

# Define a mapping for months to ensure correct ordering
month_order = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}

# Add a numerical month column for sorting
df_monthly['MONTH_NUM'] = df_monthly['MONTH'].map(month_order)

# Filter for IA and SD, and sort by YEAR and MONTH_NUM
df_filtered = df_monthly[df_monthly['BENE_STATE_ABRVTN'].isin(['IA', 'SD'])].sort_values(['YEAR', 'MONTH_NUM'])

# Get December data for each year (last month)
df_december = df_filtered[df_filtered['MONTH'] == 'December']

# Sum TOT_BENES and DUAL_TOT_BENES for IA and SD for each year
df_yearly = df_december.groupby('YEAR')[['TOT_BENES', 'DUAL_TOT_BENES']].sum().reset_index()

# Calculate YoY growth rates
df_yearly['TOT_BENES_GROWTH'] = df_yearly['TOT_BENES'].pct_change() * 100
df_yearly['DUAL_TOT_BENES_GROWTH'] = df_yearly['DUAL_TOT_BENES'].pct_change() * 100

# Fill NaN for first year with 0 (no previous year to compare)
df_yearly['TOT_BENES_GROWTH'] = df_yearly['TOT_BENES_GROWTH'].fillna(0)
df_yearly['DUAL_TOT_BENES_GROWTH'] = df_yearly['DUAL_TOT_BENES_GROWTH'].fillna(0)

# Convert YEAR to int to ensure proper ordering
df_yearly['YEAR'] = df_yearly['YEAR'].astype(int)

# Print the shape of the dataframe to check data availability
print(f"DataFrame shape: {df_yearly.shape}")
print("First few rows of the dataframe:")
print(df_yearly.head())

# Check if we have enough data points for ARIMA
if len(df_yearly) < 3:
    print("WARNING: Not enough data points for reliable ARIMA forecasting. Need at least 3 years of data.")
    # Create a placeholder forecast based on average growth
    tot_benes_future_value = df_yearly['TOT_BENES'].iloc[-1] * (1 + df_yearly['TOT_BENES_GROWTH'].mean()/100)
    dual_benes_future_value = df_yearly['DUAL_TOT_BENES'].iloc[-1] * (1 + df_yearly['DUAL_TOT_BENES_GROWTH'].mean()/100)
    tot_benes_future_growth = df_yearly['TOT_BENES_GROWTH'].mean()
    dual_benes_future_growth = df_yearly['DUAL_TOT_BENES_GROWTH'].mean()
else:
    try:
        # ARIMA Modeling for TOT_BENES
        # --------------------------
        tot_benes_model = ARIMA(df_yearly['TOT_BENES'].values, order=(1,1,0))
        tot_benes_fit = tot_benes_model.fit()

        # Forecast for 2025
        tot_benes_forecast = tot_benes_fit.forecast(steps=1)
        tot_benes_future_value = tot_benes_forecast[0]

        # Calculate forecasted growth rate
        tot_benes_future_growth = ((tot_benes_future_value / df_yearly['TOT_BENES'].values[-1]) - 1) * 100

        # ARIMA Modeling for DUAL_TOT_BENES
        # --------------------------
        dual_benes_model = ARIMA(df_yearly['DUAL_TOT_BENES'].values, order=(1,1,0))
        dual_benes_fit = dual_benes_model.fit()

        # Forecast for 2025
        dual_benes_forecast = dual_benes_fit.forecast(steps=1)
        dual_benes_future_value = dual_benes_forecast[0]

        # Calculate forecasted growth rate
        dual_benes_future_growth = ((dual_benes_future_value / df_yearly['DUAL_TOT_BENES'].values[-1]) - 1) * 100

        print("ARIMA models fitted successfully")

    except Exception as e:
        print(f"Error in ARIMA modeling: {e}")
        print("Using simple trend-based forecast instead")
        # Fallback to simple growth rate average if ARIMA fails
        tot_benes_future_value = df_yearly['TOT_BENES'].iloc[-1] * (1 + df_yearly['TOT_BENES_GROWTH'].mean()/100)
        dual_benes_future_value = df_yearly['DUAL_TOT_BENES'].iloc[-1] * (1 + df_yearly['DUAL_TOT_BENES_GROWTH'].mean()/100)
        tot_benes_future_growth = df_yearly['TOT_BENES_GROWTH'].mean()
        dual_benes_future_growth = df_yearly['DUAL_TOT_BENES_GROWTH'].mean()

# Create a new row for 2025 forecast
future_year = int(max(df_yearly['YEAR'])) + 1
future_data = {
    'YEAR': future_year,
    'TOT_BENES': tot_benes_future_value,
    'DUAL_TOT_BENES': dual_benes_future_value,
    'TOT_BENES_GROWTH': tot_benes_future_growth,
    'DUAL_TOT_BENES_GROWTH': dual_benes_future_growth
}

# Append forecast to dataframe
df_forecast = pd.concat([df_yearly, pd.DataFrame([future_data])], ignore_index=True)

# Create a color mask to differentiate historical vs forecasted data
years = df_forecast['YEAR'].tolist()
tot_benes_colors = ['blue'] * len(df_yearly) + ['rgba(0, 0, 255, 0.5)']  # Last point semi-transparent
dual_benes_colors = ['green'] * len(df_yearly) + ['rgba(0, 128, 0, 0.5)']  # Last point semi-transparent

# Create a figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add trace for total beneficiaries with forecasted point
fig.add_trace(
    go.Scatter(
        x=df_forecast['YEAR'],
        y=df_forecast['TOT_BENES'],
        mode='lines+markers+text',
        name='Total Beneficiaries (IA + SD)',
        marker=dict(
            color=tot_benes_colors,
            size=[10] * len(df_yearly) + [12],  # Make forecast point slightly larger
            line=dict(width=[1] * len(df_yearly) + [2], color='blue')
        ),
        line=dict(color='blue', width=2),
        text=[f"{growth:.1f}%" for growth in df_forecast['TOT_BENES_GROWTH']],
        textposition='top center',
        textfont=dict(color='blue', size=10)
    ),
    secondary_y=False
)

# Add trace for dual total beneficiaries with forecasted point
fig.add_trace(
    go.Scatter(
        x=df_forecast['YEAR'],
        y=df_forecast['DUAL_TOT_BENES'],
        mode='lines+markers+text',
        name='Dual Total Beneficiaries (IA + SD)',
        marker=dict(
            color=dual_benes_colors,
            size=[10] * len(df_yearly) + [12],  # Make forecast point slightly larger
            line=dict(width=[1] * len(df_yearly) + [2], color='green')
        ),
        line=dict(color='green', width=2),
        text=[f"{growth:.1f}%" for growth in df_forecast['DUAL_TOT_BENES_GROWTH']],
        textposition='top center',
        textfont=dict(color='green', size=10)
    ),
    secondary_y=True
)

# Add annotation for forecast
fig.add_annotation(
    x=future_year,
    y=tot_benes_future_value,
    text="2025 Forecast",
    showarrow=True,
    arrowhead=2,
    ax=0,
    ay=-40,
    bgcolor="rgba(255, 255, 255, 0.8)",
    bordercolor="blue",
    borderwidth=1
)

# Update layout with titles and formatting
fig.update_layout(
    title='Total and Dual Total Beneficiaries with YoY Growth and 2025 Forecast (IA + SD)',
    title_font=dict(size=16),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5),
    hovermode="x unified",
    plot_bgcolor='white',
    xaxis=dict(
        title='Year',
        titlefont=dict(size=12),
        showgrid=True,
        gridcolor='lightgray',
        tickmode='array',
        tickvals=df_forecast['YEAR']
    ),
    yaxis=dict(
        title='Total Beneficiaries',
        titlefont=dict(color='blue', size=12),
        showgrid=True,
        gridcolor='lightgray',
        tickformat=',d'  # Format with commas
    ),
    yaxis2=dict(
        title='Dual Total Beneficiaries',
        titlefont=dict(color='green', size=12),
        showgrid=False,
        tickformat=',d'  # Format with commas
    )
)

# Add grid lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', secondary_y=False)

# Add a shape to visually separate historical and forecast data
fig.add_shape(
    type="line",
    x0=df_yearly['YEAR'].iloc[-1] + 0.5,
    y0=0,
    x1=df_yearly['YEAR'].iloc[-1] + 0.5,
    y1=1,
    yref="paper",
    line=dict(color="gray", width=1, dash="dot"),
)

# Show the plot
fig.show()

# Print the yearly data including forecast
print("\nHistorical and Forecasted Data:")
print(df_forecast[['YEAR', 'TOT_BENES', 'DUAL_TOT_BENES', 'TOT_BENES_GROWTH', 'DUAL_TOT_BENES_GROWTH']])

DataFrame shape: (12, 5)
First few rows of the dataframe:
   YEAR  TOT_BENES  DUAL_TOT_BENES  TOT_BENES_GROWTH  DUAL_TOT_BENES_GROWTH
0  2013     704641          101338          0.000000               0.000000
1  2014     721087          102474          2.333954               1.121001
2  2015     736229          102977          2.099885               0.490856
3  2016     754880          104376          2.533315               1.358556
4  2017     770963          104033          2.130537              -0.328620
ARIMA models fitted successfully



Historical and Forecasted Data:
    YEAR      TOT_BENES  DUAL_TOT_BENES  TOT_BENES_GROWTH  \
0   2013  704641.000000   101338.000000          0.000000   
1   2014  721087.000000   102474.000000          2.333954   
2   2015  736229.000000   102977.000000          2.099885   
3   2016  754880.000000   104376.000000          2.533315   
4   2017  770963.000000   104033.000000          2.130537   
5   2018  792368.000000   105019.000000          2.776398   
6   2019  809618.000000   105522.000000          2.177019   
7   2020  823277.000000   109582.000000          1.687092   
8   2021  838781.000000   114510.000000          1.883206   
9   2022  856135.000000   119684.000000          2.068955   
10  2023  873961.000000   108781.000000          2.082148   
11  2024  889946.000000   105523.000000          1.829029   
12  2025  890471.222962   105475.904305          0.059017   

    DUAL_TOT_BENES_GROWTH  
0                0.000000  
1                1.121001  
2                0.490856  


# ARIMA Model Forecast for Medicare Beneficiaries (IA + SD): 2013-2025

## Model Selection and Methodology

### Why ARIMA?
For this time series forecasting task, we selected an ARIMA (AutoRegressive Integrated Moving Average) model for several reasons:

1. **Time Series Nature**: The Medicare beneficiary data represents a classic time series with yearly observations from 2013-2024, making ARIMA a natural choice.

2. **Trend Component**: The data shows a clear upward trend in total beneficiaries over time, which ARIMA can capture through differencing (the "I" component).

3. **Statistical Foundation**: ARIMA provides a well-established statistical framework with interpretable parameters and confidence intervals.

4. **Simplicity and Interpretability**: While more complex models exist, ARIMA provides a good balance between accuracy and simplicity for forecasting one year ahead.

### Model Parameters

We selected an ARIMA(1,1,0) model for both Total Beneficiaries and Dual Total Beneficiaries time series:

- **p=1**: First-order autoregressive component, suggesting that each year's value depends on the previous year's value
- **d=1**: First-order differencing to handle the upward trend and make the series stationary
- **q=0**: No moving average component, keeping the model simpler and more stable

The model specification ARIMA(1,1,0) is equivalent to a first-order differenced autoregressive model, which is appropriate for data with a consistent trend pattern like we observe in the Medicare beneficiary counts.

## Implementation Details

The forecasting process involved:

1. **Data Preparation**:
   - Filtered for Iowa (IA) and South Dakota (SD) data
   - Aggregated to yearly totals based on December values
   - Calculated year-over-year growth rates

2. **Model Fitting**:
   - Fit separate ARIMA(1,1,0) models for Total Beneficiaries and Dual Total Beneficiaries
   - Used maximum likelihood estimation to determine model parameters

3. **Forecasting**:
   - Generated point forecasts for 2025
   - Calculated projected growth rates based on the forecasts

4. **Robustness Measures**:
   - Implemented fallback methods using average growth rates in case of model failure
   - Added data validation to ensure sufficient historical data

## Results Analysis

### Forecast Results

| Metric | 2024 (Actual) | 2025 (Forecast) | Projected Growth |
|--------|---------------|-----------------|------------------|
| Total Beneficiaries | 889,946 | 890,471 | 0.059% |
| Dual Total Beneficiaries | 105,523 | 105,476 | -0.045% |

### Key Observations

1. **Total Beneficiaries**:
   - The model predicts minimal growth (0.059%) in total beneficiaries for 2025
   - This represents a significant slowdown compared to the historical average growth rate of ~2% per year
   - The forecast suggests the Total Beneficiaries count is reaching a plateau

2. **Dual Total Beneficiaries**:
   - The model predicts a very slight decrease (-0.045%) in dual beneficiaries for 2025
   - This continues the declining trend observed in 2022-2024
   - The forecast aligns with the recent negative growth pattern seen in this metric

### Historical Context

The historical data reveals several notable patterns:

1. **Total Beneficiaries** showed consistent positive growth throughout 2013-2024, ranging from 1.69% to 2.78% per year.

2. **Dual Total Beneficiaries** showed more volatility:
   - Moderate growth (0.5-1.4%) from 2013-2018
   - Stronger growth (3.8-4.5%) from 2019-2022
   - Sharp decline (-9.1%) in 2023
   - Continued decline (-3.0%) in 2024

The model captures these trends and suggests that while total beneficiary growth is slowing to near-zero, the dual beneficiary count is stabilizing after two years of significant declines.

## Model Limitations

1. **Limited Historical Data**: The model is based on only 12 years of data (2013-2024), which may limit its predictive power.

2. **Policy Sensitivity**: Medicare enrollment is highly sensitive to policy changes, which ARIMA cannot anticipate.

3. **Demographic Shifts**: The model doesn't explicitly account for demographic transitions like the aging Baby Boomer population.

4. **External Factors**: Economic conditions, healthcare access changes, and other external factors aren't directly incorporated.

## Conclusion

The ARIMA(1,1,0) forecast suggests that Medicare beneficiary growth in Iowa and South Dakota is likely to plateau in 2025, with total beneficiaries showing minimal growth and dual beneficiaries stabilizing after recent declines. The forecast aligns with the recent trends in the data and provides a statistically sound projection based on available historical information.

For increased forecast reliability, future work could explore:
- Seasonal ARIMA models if monthly data is available
- Multivariate models incorporating demographic projections
- Ensemble approaches combining multiple forecasting methods

In [ ]:
# Filter for most recent years and relevant columns
df_bed_phys_recent = df_bed_phys[df_bed_phys['YEAR'].isin([2023, 2024])][['YEAR', 'STATE', 'BED_CNT', 'PHYSN_CNT']]

# Pivot to compare 2023 and 2024
df_bed_phys_pivot = df_bed_phys_recent.pivot(index='STATE', columns='YEAR', values=['BED_CNT', 'PHYSN_CNT'])
print(df_bed_phys_pivot)

        BED_CNT           PHYSN_CNT         
YEAR       2023      2024      2023     2024
STATE                                       
IA     427040.0  423499.0   7344.87  7179.53
SD     120086.0  120879.0   4124.63  4122.00


In [ ]:
# Get the growth rate from your forecast (let's assume this is total beneficiaries growth)
growth_rate = tot_benes_future_growth / 100  # Convert from percentage

# Calculate 2025 projections for each state
projections = []
for state in ['IA', 'SD']:
    # Get most recent data (2024)
    bed_2024 = df_bed_phys[df_bed_phys['YEAR'] == 2024][df_bed_phys['STATE'] == state]['BED_CNT'].values[0]
    phys_2024 = df_bed_phys[df_bed_phys['YEAR'] == 2024][df_bed_phys['STATE'] == state]['PHYSN_CNT'].values[0]

    # Project 2025 values
    bed_2025 = bed_2024 * (1 + growth_rate)
    phys_2025 = phys_2024 * (1 + growth_rate)

    projections.append({
        'YEAR': 2025,
        'STATE': state,
        'BED_CNT': round(bed_2025, 2),
        'PHYSN_CNT': round(phys_2025, 2)
    })

# Create dataframe with projections
df_2025_projections = pd.DataFrame(projections)
print("\n2025 Projections:")
print(df_2025_projections)


2025 Projections:
   YEAR STATE    BED_CNT  PHYSN_CNT
0  2025    IA  423748.94    7183.77
1  2025    SD  120950.34    4124.43


In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('Hospital Beds Projection', 'Physicians Projection'))

for i, state in enumerate(['IA', 'SD']):
    state_data = df_bed_phys[df_bed_phys['STATE'] == state].sort_values('YEAR')
    proj = df_2025_projections[df_2025_projections['STATE'] == state]

    # Beds plot
    fig.add_trace(
        go.Scatter(
            x=state_data['YEAR'],
            y=state_data['BED_CNT'],
            mode='lines+markers',
            name=f'{state} Beds',
            line=dict(dash='solid'),
            marker=dict(size=8)
        ),
        row=1, col=1
    )

    # Add projection
    fig.add_trace(
        go.Scatter(
            x=[2024, 2025],
            y=[state_data[state_data['YEAR'] == 2024]['BED_CNT'].values[0], proj['BED_CNT'].values[0]],
            mode='lines+markers',
            name=f'{state} Projected',
            line=dict(dash='dot'),
            marker=dict(symbol='diamond-open')
        ),
        row=1, col=1
    )

    # Physicians plot
    fig.add_trace(
        go.Scatter(
            x=state_data['YEAR'],
            y=state_data['PHYSN_CNT'],
            mode='lines+markers',
            name=f'{state} Physicians',
            line=dict(dash='solid'),
            marker=dict(size=8)
        ),
        row=1, col=2
    )

    # Add projection
    fig.add_trace(
        go.Scatter(
            x=[2024, 2025],
            y=[state_data[state_data['YEAR'] == 2024]['PHYSN_CNT'].values[0], proj['PHYSN_CNT'].values[0]],
            mode='lines+markers',
            showlegend=False,
            line=dict(dash='dot'),
            marker=dict(symbol='diamond-open')
        ),
        row=1, col=2
    )

fig.update_layout(
    title='Projected Healthcare Resource Needs for 2025',
    hovermode='x unified',
    height=500
)

fig.show()

In [ ]:
spending_df = pd.read_csv('/content/drive/MyDrive/Wellmark Case/Medicare Cleaned Data/spending_per_beneficiary_2014-2022.csv')

In [ ]:
spending_df.head(1000)

,YEAR,STATE,SPENDING_PER_BENE
0,2014,IA,6121.55
1,2014,IA,6797.98
2,2014,IA,6014.50
3,2014,SD,5723.34
4,2014,SD,6940.99
5,2014,SD,5549.91
6,2015,IA,6325.74
7,2015,IA,7135.08
8,2015,IA,6200.25
9,2015,SD,5792.13


In [ ]:
import pandas as pd
import plotly.graph_objects as go
benefit_columns = ['A_TOT_BENES', 'B_TOT_BENES']
required_cols = benefit_columns + ['DUAL_TOT_BENES', 'YEAR']
missing_cols = [col for col in required_cols if col not in df_yearly.columns]
if missing_cols:
    print(f"Error: The following columns are missing from your Excel file: {missing_cols}")
else:
    grouped = df_yearly.groupby('YEAR').agg({**{col: 'sum' for col in benefit_columns}, 'DUAL_TOT_BENES': 'sum'})
    fig = go.Figure()

    # Iterate over each year to create traces
    for year in grouped.index:
        # Total beneficiaries for A and B for this year
        benefit_totals = grouped.loc[year, benefit_columns].values
        # Dual-eligible beneficiaries (use DUAL_TOT_BENES directly as a total)
        dual_total = grouped.loc[year, 'DUAL_TOT_BENES']

        # Add trace for A_TOT_BENES
        fig.add_trace(go.Bar(
            x=benefit_columns,
            y=[benefit_totals[0], 0],  # Only A_TOT_BENES value for its position
            name=f'A Beneficiaries ({year})',
            marker_color='skyblue',
            width=0.3,
            visible=(year == grouped.index[0]),  # Only first year visible by default
            offsetgroup=0
        ))

        # Add trace for B_TOT_BENES
        fig.add_trace(go.Bar(
            x=benefit_columns,
            y=[0, benefit_totals[1]],  # Only B_TOT_BENES value for its position
            name=f'B Beneficiaries ({year})',
            marker_color='lightgreen',
            width=0.3,
            visible=(year == grouped.index[0]),  # Only first year visible by default
            offsetgroup=1
        ))

        # Add trace for DUAL_TOT_BENES
        fig.add_trace(go.Bar(
            x=benefit_columns,
            y=[dual_total, dual_total],  # Show dual total consistently (or adjust if split)
            name=f'Dual-Eligible Beneficiaries ({year})',
            marker_color='lightcoral',
            width=0.3,
            visible=(year == grouped.index[0]),  # Only first year visible by default
            offsetgroup=2
        ))
    buttons = []
    for i, year in enumerate(grouped.index):
        buttons.append(dict(
            label=str(year),
            method='update',
            args=[{'visible': [False] * len(fig.data)}]  # Reset all to invisible
        ))
        # Set visibility for this year's traces (3 traces per year: A, B, Dual)
        buttons[-1]['args'][0]['visible'][i * 3] = True      # A_TOT_BENES
        buttons[-1]['args'][0]['visible'][i * 3 + 1] = True  # B_TOT_BENES
        buttons[-1]['args'][0]['visible'][i * 3 + 2] = True  # DUAL_TOT_BENES
    fig.update_layout(
        title='Trend of Beneficiaries (A, B, and Dual-Eligible) by Year',
        xaxis_title='Benefit Category',
        yaxis_title='Number of Beneficiaries',
        xaxis={'tickangle': -45},
        barmode='group',
        bargap=0.2,
        bargroupgap=0.1,
        yaxis=dict(gridcolor='lightgray', gridwidth=1),
        legend=dict(title='Category'),
        width=1200,
        height=800,
        updatemenus=[
            dict(
                buttons=buttons,
                direction='down',
                showactive=True,
                x=0.1,
                xanchor='left',
                y=1.1,
                yanchor='top'
            )
        ]
    )
    fig.show()
    print("Trend Analysis by Year:")
    for year in grouped.index:
        a_tot = grouped.loc[year, 'A_TOT_BENES']
        b_tot = grouped.loc[year, 'B_TOT_BENES']
        dual_tot = grouped.loc[year, 'DUAL_TOT_BENES']
        print(f"Year {year}:")
        print(f"  - A Total Beneficiaries: {a_tot}")
        print(f"  - B Total Beneficiaries: {b_tot}")
        print(f"  - Dual-Eligible Beneficiaries: {dual_tot}")
        print(f"  - A vs B Comparison: {'A > B' if a_tot > b_tot else 'B > A' if b_tot > a_tot else 'A = B'}")

Trend Analysis by Year:
Year 2013:
  - A Total Beneficiaries: 696071
  - B Total Beneficiaries: 650743
  - Dual-Eligible Beneficiaries: 100540
  - A vs B Comparison: A > B
Year 2014:
  - A Total Beneficiaries: 712241
  - B Total Beneficiaries: 664894
  - Dual-Eligible Beneficiaries: 101987
  - A vs B Comparison: A > B
Year 2015:
  - A Total Beneficiaries: 727453
  - B Total Beneficiaries: 678676
  - Dual-Eligible Beneficiaries: 102614
  - A vs B Comparison: A > B
Year 2016:
  - A Total Beneficiaries: 745274
  - B Total Beneficiaries: 694391
  - Dual-Eligible Beneficiaries: 103868
  - A vs B Comparison: A > B
Year 2017:
  - A Total Beneficiaries: 763248
  - B Total Beneficiaries: 709755
  - Dual-Eligible Beneficiaries: 104472
  - A vs B Comparison: A > B
Year 2018:
  - A Total Beneficiaries: 783044
  - B Total Beneficiaries: 727497
  - Dual-Eligible Beneficiaries: 104793
  - A vs B Comparison: A > B
Year 2019:
  - A Total Beneficiaries: 800485
  - B Total Beneficiaries: 743224
  - Dual-

In [ ]:
import pandas as pd
import plotly.graph_objects as go

age_columns = [
    'AGE_LT_25_BENES', 'AGE_25_TO_44_BENES', 'AGE_45_TO_64_BENES',
    'AGE_65_TO_69_BENES', 'AGE_70_TO_74_BENES', 'AGE_75_TO_79_BENES',
    'AGE_80_TO_84_BENES', 'AGE_85_TO_89_BENES', 'AGE_90_TO_94_BENES',
    'AGE_GT_94_BENES'
]

required_cols = age_columns + ['DUAL_TOT_BENES', 'YEAR']
missing_cols = [col for col in required_cols if col not in df_yearly.columns]
if missing_cols:
    print(f"Error: The following columns are missing from your Excel file: {missing_cols}")
else:
    grouped = df_yearly.groupby('YEAR').agg({**{col: 'sum' for col in age_columns}, 'DUAL_TOT_BENES': 'sum'})
    fig = go.Figure()

    # Iterate over each year to create traces
    for year in grouped.index:
        # Total beneficiaries for this year
        age_totals = grouped.loc[year, age_columns].values
        # Dual-eligible beneficiaries (hypothetical split from DUAL_TOT_BENES)
        dual_total = grouped.loc[year, 'DUAL_TOT_BENES']
        dual_per_age = age_totals * (dual_total / age_totals.sum() if age_totals.sum() > 0 else 0)  # Proportional split

        # Add trace for Total Beneficiaries
        fig.add_trace(go.Bar(
            x=age_columns,
            y=age_totals,
            name=f'Total Beneficiaries ({year})',
            marker_color='skyblue',
            width=0.4,
            visible=(year == grouped.index[0]),  # Only first year visible by default
            offsetgroup=0  # Group for total beneficiaries
        ))

        # Add trace for Dual-Eligible Beneficiaries
        fig.add_trace(go.Bar(
            x=age_columns,
            y=dual_per_age,
            name=f'Dual-Eligible Beneficiaries ({year})',
            marker_color='lightcoral',
            width=0.4,
            visible=(year == grouped.index[0]),  # Only first year visible by default
            offsetgroup=1  # Group for dual-eligible
        ))

    buttons = []
    for i, year in enumerate(grouped.index):
        buttons.append(dict(
            label=str(year),
            method='update',
            args=[{'visible': [False] * len(fig.data)}]  # Reset all to invisible
        ))
        # Set visibility for this year's traces (2 traces per year: total and dual)
        buttons[-1]['args'][0]['visible'][i * 2] = True  # Total beneficiaries
        buttons[-1]['args'][0]['visible'][i * 2 + 1] = True  # Dual-eligible

    # Update layout with dropdown
    fig.update_layout(
        title='Trend of Beneficiaries by Age Group with Dual-Eligible by Year',
        xaxis_title='Age Group',
        yaxis_title='Number of Beneficiaries',
        xaxis={'tickangle': -45},
        barmode='group',
        bargap=0.2,
        bargroupgap=0.1,
        yaxis=dict(gridcolor='lightgray', gridwidth=1),
        legend=dict(title='Category'),
        width=1000,
        height=600,
        updatemenus=[
            dict(
                buttons=buttons,
                direction='down',
                showactive=True,
                x=0.1,
                xanchor='left',
                y=1.1,
                yanchor='top'
            )
        ]
    )

    # Display the chart
    fig.show()
    print("Trend Analysis by Year:")
    for year in grouped.index:
        age_totals = grouped.loc[year, age_columns]
        dual_per_age = age_totals * (grouped.loc[year, 'DUAL_TOT_BENES'] / age_totals.sum() if age_totals.sum() > 0 else 0)
        peak_age = age_totals.idxmax()
        dual_peak_age = dual_per_age.idxmax()
        print(f"Year {year}:")
        print(f"  - Peak age group for total beneficiaries: {peak_age}")
        print(f"  - Peak age group for dual-eligible beneficiaries: {dual_peak_age}")
        print(f"  - Total Dual-Eligible Beneficiaries: {grouped.loc[year, 'DUAL_TOT_BENES']}")

Trend Analysis by Year:
Year 2013:
  - Peak age group for total beneficiaries: AGE_65_TO_69_BENES
  - Peak age group for dual-eligible beneficiaries: AGE_65_TO_69_BENES
  - Total Dual-Eligible Beneficiaries: 100540
Year 2014:
  - Peak age group for total beneficiaries: AGE_65_TO_69_BENES
  - Peak age group for dual-eligible beneficiaries: AGE_65_TO_69_BENES
  - Total Dual-Eligible Beneficiaries: 101987
Year 2015:
  - Peak age group for total beneficiaries: AGE_65_TO_69_BENES
  - Peak age group for dual-eligible beneficiaries: AGE_65_TO_69_BENES
  - Total Dual-Eligible Beneficiaries: 102614
Year 2016:
  - Peak age group for total beneficiaries: AGE_65_TO_69_BENES
  - Peak age group for dual-eligible beneficiaries: AGE_65_TO_69_BENES
  - Total Dual-Eligible Beneficiaries: 103868
Year 2017:
  - Peak age group for total beneficiaries: AGE_65_TO_69_BENES
  - Peak age group for dual-eligible beneficiaries: AGE_65_TO_69_BENES
  - Total Dual-Eligible Beneficiaries: 104472
Year 2018:
  - Peak a

In [ ]:
df3 = pd.read_csv('/content/drive/MyDrive/Wellmark Case/Medicare Cleaned Data/ia_sd_cleaned.csv')

In [ ]:
df3.columns

Index(['zip', 'lat', 'lng', 'city', 'state_id', 'state_name', 'zcta',
       'parent_zcta', 'population', 'density', 'county_fips', 'county_name',
       'county_weights', 'imprecise', 'military', 'timezone', 'age_median',
       'age_under_10', 'age_10_to_19', 'age_20s', 'age_30s', 'age_40s',
       'age_50s', 'age_60s', 'age_70s', 'age_over_80', 'age_over_65',
       'age_18_to_24', 'age_over_18', 'male', 'female', 'married', 'divorced',
       'never_married', 'widowed', 'family_size', 'family_dual_income',
       'income_household_median', 'income_household_under_5',
       'income_household_5_to_10', 'income_household_10_to_15',
       'income_household_15_to_20', 'income_household_20_to_25',
       'income_household_25_to_35', 'income_household_35_to_50',
       'income_household_50_to_75', 'income_household_75_to_100',
       'income_household_100_to_150', 'income_household_150_over',
       'income_household_six_figure', 'income_individual_median',
       'home_ownership', 'hou

In [ ]:
# Import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create the dataframe as before
yearly_high_need = df_yearly.groupby('YEAR')[['AGED_ESRD_BENES', 'DSBLD_TOT_BENES']].sum().reset_index()
yearly_high_need['ESRD_Proportion'] = yearly_high_need['AGED_ESRD_BENES'] / df_yearly.groupby('YEAR')['DUAL_TOT_BENES'].sum()

# Create a plotly figure
fig = go.Figure()

# Add traces for each line
fig.add_trace(go.Scatter(
    x=yearly_high_need['YEAR'],
    y=yearly_high_need['AGED_ESRD_BENES'],
    mode='lines+markers',
    name='ESRD Aged',
    marker=dict(symbol='circle')
))

fig.add_trace(go.Scatter(
    x=yearly_high_need['YEAR'],
    y=yearly_high_need['DSBLD_TOT_BENES'],
    mode='lines+markers',
    name='Disabled',
    marker=dict(symbol='circle')
))

# Update layout with legend on the right side
fig.update_layout(
    title='High-Need Subgroups Over Time',
    xaxis_title='Year',
    yaxis_title='Beneficiaries',
    legend=dict(
        x=1.02,
        y=0.5,
        xanchor='left',
        yanchor='middle'
    ),
    template='plotly_white',
    width=800,
    height=500,
    grid=dict(rows=1, columns=1),
    margin=dict(r=120)  # Add right margin to accommodate the legend
)

# Add grid
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Show the plot
fig.show()